<a href="https://colab.research.google.com/github/macbition1/ROC1/blob/main/ROS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch==1.7.0
!pip install torchvision==0.8.0
!pip install tensorflow-gpu==1.15.0
!pip install scipy==1.5.2
!pip install Pillow==6.2.2
!pip install numpy==1.17.0
!pip install scikit-learn==0.22.1

     |████████████████████████████████| 776.7 MB 4.2 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:

      Successfully uninstalled torch-1.10.0+cu111

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.7.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.7.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.7.0 which is incompatible.
     |████████████████████████████████| 11.8 MB 4.7 MB/s 
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
     |████████████████████████████████| 411.5 MB 7.8 kB/s 
   

     |████████████████████████████████| 20.3 MB 1.0 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.7.0 which is incompatible.
tensorflow 2.7.0 requires tensorboard~=2.6, but you have tensorboard 1.15.0 which is incompatible.
tensorflow 2.7.0 requires tensorflow-estimator<2.8,~=2.7.0rc0, but you have tensorflow-estimator 1.15.1 which is incompatible.
tensorflow-probability 0.15.0 requires gast>=0.3.2, but you have gast 0.2.2 which is incompatible.
pywavelets 1.2.0 requires numpy>=1.17.3, but you have numpy 1.17.0 which is incompatible.
kapre 0.3.6 requires numpy>=1.18.5, but you have numpy 1.17.0 which is incompatible.
jaxlib 0.1.71+cuda111 requi

     |████████████████████████████████| 7.0 MB 5.5 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.6 requires numpy>=1.18.5, but you have numpy 1.17.0 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.22.1 which is incompatible.


In [ ]:
#data_helper
import torch
from torchvision import transforms

from dataset import Dataset, TestDataset, _dataset_info


def get_train_dataloader(args,txt_file):


    img_transformer = get_train_transformers(args)
    name_train, labels_train = _dataset_info(txt_file)
    train_dataset = Dataset(name_train, labels_train, args.path_dataset, img_transformer=img_transformer)
    loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=4, pin_memory=True, drop_last=True)

    return loader


def get_val_dataloader(args,txt_file):

    names, labels = _dataset_info(txt_file)
    img_tr = get_test_transformer(args)
    test_dataset = TestDataset(names, labels,args.path_dataset, img_transformer=img_tr)
    loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4, pin_memory=True, drop_last=False)

    return loader


def get_train_transformers(args):

    img_tr = [transforms.RandomResizedCrop((int(args.image_size), int(args.image_size)), (args.min_scale, args.max_scale))]

    if args.jitter > 0.0:
        img_tr.append(transforms.ColorJitter(brightness=args.jitter, contrast=args.jitter, saturation=args.jitter, hue=min(0.5, args.jitter)))
    if args.random_grayscale:
        img_tr.append(transforms.RandomGrayscale(args.random_grayscale))

    img_tr = img_tr + [transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]

    return transforms.Compose(img_tr)


def get_test_transformer(args):

    img_tr = [transforms.Resize((args.image_size, args.image_size)), transforms.ToTensor(),
              transforms.Normalize([0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]

    return transforms.Compose(img_tr)

ModuleNotFoundError: ignored

In [ ]:
#dataset
import torch.utils.data as data
from PIL import Image
from random import random
import random
import torchvision.transforms.functional as TF



def _dataset_info(txt_labels):
    with open(txt_labels, 'r') as f:
        images_list = f.readlines()

    file_names = []
    labels = []
    for row in images_list:
        row = row.split(' ')
        file_names.append(row[0])
        labels.append(int(row[1]))

    return file_names, labels


class Dataset(data.Dataset):
    def __init__(self, names, labels, path_dataset,img_transformer=None):
        self.data_path = path_dataset
        self.names = names
        self.labels = labels
        self._image_transformer = img_transformer

    def __getitem__(self, index):


        return img, int(self.labels[index]), img_rot, index_rot

    def __len__(self):
        return len(self.names)



class TestDataset(data.Dataset):
    def __init__(self, names, labels, path_dataset,img_transformer=None):
        self.data_path = path_dataset
        self.names = names
        self.labels = labels
        self._image_transformer = img_transformer

    def __getitem__(self, index):


        return img, int(self.labels[index]), img_rot, index_rot

    def __len__(self):
        return len(self.names)



In [ ]:
#eval_target

import torch
import numpy as np
from sklearn.metrics import roc_auc_score
import random


#### Implement the evaluation on the target for the known/unknown separation

def evaluation(args,feature_extractor,rot_cls,target_loader_eval,device):

    feature_extractor.eval()
    rot_cls.eval()


    with torch.no_grad():
        for it, (data,class_l,data_rot,rot_l) in enumerate(target_loader_eval):
            data, class_l, data_rot, rot_l = data.to(device), class_l.to(device), data_rot.to(device), rot_l.to(device)



    auroc = roc_auc_score(ground_truth,normality_score)
    print('AUROC %.4f' % auroc)

    # create new txt files
    rand = random.randint(0,100000)
    print('Generated random number is :', rand)

    # This txt files will have the names of the source images and the names of the target images selected as unknown
    target_unknown = open('new_txt_list/' + args.source + '_known_' + str(rand) + '.txt','w')

    # This txt files will have the names of the target images selected as known
    target_known = open('new_txt_list/' + args.target + '_known_' + str(rand) + '.txt','w')



    print('The number of target samples selected as known is: ',number_of_known_samples)
    print('The number of target samples selected as unknown is: ', number_of_unknown_samples)

    return rand

In [ ]:
#main.py
import argparse
import os

import torch

import data_helper
from resnet import resnet18_feat_extractor, Classifier

from step1_KnownUnknownSep import step1
from eval_target import evaluation
import numpy as npy

from torch.utils.data.sampler import SubsetRandomSampler

# from step2_SourceTargetAdapt import step2


def get_args():
    parser = argparse.ArgumentParser(description="Script to launch training",
                                     formatter_class=argparse.ArgumentDefaultsHelpFormatter)

    parser.add_argument("--source", default='Art', help="Source name")
    parser.add_argument("--target", default='Clipart', help="Target name")
    parser.add_argument("--n_classes_known", type=int, default=45, help="Number of known classes")
    parser.add_argument("--n_classes_tot", type=int, default=65, help="Number of unknown classes")

    # dataset path
    parser.add_argument("--path_dataset", default="./data", help="Path where the Office-Home dataset is located")

    # data augmentation
    parser.add_argument("--min_scale", default=0.8, type=float, help="Minimum scale percent")
    parser.add_argument("--max_scale", default=1.0, type=float, help="Maximum scale percent")
    parser.add_argument("--jitter", default=0.4, type=float, help="Color jitter amount")
    parser.add_argument("--random_grayscale", default=0.1, type=float,help="Randomly greyscale the image")

    # training parameters
    parser.add_argument("--image_size", type=int, default=222, help="Image size")
    parser.add_argument("--batch_size", type=int, default=128, help="Batch size")
    parser.add_argument("--learning_rate", type=float, default=0.001, help="Learning rate")

    parser.add_argument("--epochs_step1", type=int, default=10, help="Number of epochs of step1 for known/unknown separation")
    parser.add_argument("--epochs_step2", type=int, default=10, help="Number of epochs of step2 for source-target adaptation")

    parser.add_argument("--train_all", type=bool, default=True, help="If true, all network weights will be trained")

    parser.add_argument("--weight_RotTask_step1", type=float, default=0.5, help="Weight for the rotation loss in step1")
    parser.add_argument("--weight_RotTask_step2", type=float, default=0.5, help="Weight for the rotation loss in step2")
    parser.add_argument("--threshold", type=float, default=0.5, help="Threshold for the known/unkown separation")

    # tensorboard logger
    parser.add_argument("--tf_logger", type=bool, default=True, help="If true will save tensorboard compatible logs")
    parser.add_argument("--folder_name", default=None, help="Used by the logger to save logs")

    return parser.parse_args()


class Trainer:
    def __init__(self, args):
        self.args = args

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # initialize the network with a number of classes equals to the number of known classes + 1 (the unknown class, trained only in step2)
        self.feature_extractor = resnet18_feat_extractor()
        self.obj_classifier = Classifier(512,self.args.n_classes_known+1)
        self.rot_classifier = Classifier(512*2,4)

        self.feature_extractor = self.feature_extractor.to(self.device)
        self.obj_cls = self.obj_classifier.to(self.device)
        self.rot_cls = self.rot_classifier.to(self.device)

        source_path_file = 'txt_list/'+args.source+'_known.txt'
        self.source_loader = data_helper.get_train_dataloader(args,source_path_file)
        
        
        target_path_file = 'txt_list/' + args.target + '.txt'
        self.target_loader_train = data_helper.get_val_dataloader(args,target_path_file)
        self.target_loader_eval = data_helper.get_val_dataloader(args,target_path_file)

        print("Source: ",self.args.source," Target: ",self.args.target)
        print("Dataset size: source %d, target %d" % (len(self.source_loader.dataset), len(self.target_loader_train.dataset)))

    def do_training(self):

        # just check if final training parameters are saved somewhere. If so, so not train again

        if not os.path.isfile("./feature_extractor_params.pt") and not os.path.isfile("./rot_cls_params.pt"):
            print('Step 1 --------------------------------------------')
            step1(self.args,self.feature_extractor,self.rot_cls,self.obj_cls,self.source_loader,self.device)

        print('Target - Evaluation -- for known/unknown separation')

        # if params are already computed, load the model and procede with its evaluation

        self.rot_cls.load_state_dict(torch.load("./feature_extractor_params.pt"), strict=False)
        self.feature_extractor.load_state_dict(torch.load("./rot_cls_params.pt"), strict=False)

        rand = evaluation(self.args,self.feature_extractor,self.rot_cls,self.target_loader_eval,self.device)

        # new dataloaders
        source_path_file = 'new_txt_list/' + self.args.source + '_known_'+str(rand)+'.txt'
        self.source_loader = data_helper.get_train_dataloader(self.args,source_path_file)

        target_path_file = 'new_txt_list/' + self.args.target + '_known_' + str(rand) + '.txt'
        self.target_loader_train = data_helper.get_train_dataloader(self.args,target_path_file)
        self.target_loader_eval = data_helper.get_val_dataloader(self.args,target_path_file)

        """print('Step 2 --------------------------------------------')
        step2(self.args,self.feature_extractor,self.rot_cls,self.obj_cls,self.source_loader,self.target_loader_train,self.target_loader_eval,self.device)"""

def main():
    args = get_args()
    trainer = Trainer(args)
    trainer.do_training()


if __name__ == "__main__":
    torch.backends.cudnn.benchmark = True
    main()

In [ ]:
#optimizer_helper
from torch import optim


def get_optim_and_scheduler(feature_extractor,rot_cls,obj_cls, epochs, lr, train_all):

    if train_all:
        params = list(feature_extractor.parameters()) + list(rot_cls.parameters()) + list(obj_cls.parameters())
    else:
        params = list(rot_cls.parameters()) + list(obj_cls.parameters())

    optimizer = optim.SGD(params, weight_decay=.0005, momentum=.9, lr=lr)
    step_size = int(epochs * .8)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size)


    print("Step size: %d" % step_size)
    return optimizer, scheduler

In [ ]:
#reset
from torch import nn
from torch.utils import model_zoo
from torchvision.models.resnet import BasicBlock, model_urls, Bottleneck


class ResNet(nn.Module):
    def __init__(self, block, layers):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def is_patch_based(self):
        return False

    def forward(self, x, **kwargs):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        return x


class Classifier(nn.Module):
    def __init__(self,input_size,classes):
        super(Classifier, self).__init__()
        self.class_classifier = nn.Linear(input_size, classes)

    def forward(self, x):

        return self.class_classifier(x)



def resnet18_feat_extractor():
    """Constructs a ResNet-18 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2])
    model.load_state_dict(model_zoo.load_url(model_urls['resnet18']), strict=False)

    return model


In [ ]:
#step1

import torch
from torch import nn
from optimizer_helper import get_optim_and_scheduler
from tqdm import tqdm

#### Implement Step1

"""def _do_epoch(args,feature_extractor,rot_cls,obj_cls,dataloaders,optimizer,device,phase):
    
    if phase == "train":
        print("TRAINING MODE")
        feature_extractor.train()
        obj_cls.train()
        rot_cls.train()
    else:
        print("VALIDATION MODE")
        feature_extractor.eval()
        obj_cls.eval()
        rot_cls.eval()

    running_img_corrects = 0
    running_rot_corrects = 0

    for i, (imgs, lbls, rot_imgs, rot_lbls) in tqdm(enumerate(dataloaders[phase])):
        imgs = imgs.to(device)
        lbls = lbls.to(device)
        rot_imgs = rot_imgs.to(device)
        rot_lbls = rot_lbls.to(device)

        # forward
        with torch.set_grad_enabled(phase == "train"):
            imgs_out = feature_extractor(imgs)
            imgs_predictions = obj_cls(imgs_out)
            
            rot_out = feature_extractor(rot_imgs)
            rot_predictions = rot_cls(torch.cat((rot_out, imgs_out), dim=1))
            
            _, imgs_preds = torch.max(imgs_predictions, 1)
            _, rot_preds = torch.max(rot_predictions, 1)

            # compute loss

            img_loss = nn.CrossEntropyLoss(imgs_predictions, lbls)
            rot_loss = nn.CrossEntropyLoss(rot_predictions, rot_lbls)

            loss = img_loss + args.weight_RotTask_step1*rot_loss

            if phase == "train":
                    # compute gradient + update params
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            # statistics
            # raccolgo in ciascun batch la loss per quel batch e quanti elementi sono stati
            # classificati correttamente

        #running_loss += loss.item() * imgs.size(0)
        running_img_corrects += torch.sum(imgs_preds == lbls.data)
        running_rot_corrects += torch.sum(rot_preds == rot_lbls.data)

    #class_loss = running_loss / dataset_sizes[phase]
    img_acc = (running_img_corrects.double() / len(dataloaders[phase].dataset)) * 100
    rot_acc = (running_rot_corrects.double() / len(dataloaders[phase].dataset)) * 100

    return img_loss, img_acc, rot_loss, rot_acc"""

def _do_epoch(args,feature_extractor,rot_cls,obj_cls,source_loader,optimizer,device,criterion):
    
    feature_extractor.train()
    obj_cls.train()
    rot_cls.train()
    
    running_img_corrects = 0
    running_rot_corrects = 0

    for i, (imgs, lbls, rot_imgs, rot_lbls) in tqdm(enumerate(source_loader)):
        imgs = imgs.to(device)
        lbls = lbls.to(device)
        rot_imgs = rot_imgs.to(device)
        rot_lbls = rot_lbls.to(device)

        # forward
        imgs_out = feature_extractor(imgs)
        imgs_predictions = obj_cls(imgs_out)

        rot_out = feature_extractor(rot_imgs)
        rot_predictions = rot_cls(torch.cat((rot_out, imgs_out), dim=1))

        _, imgs_preds = torch.max(imgs_predictions, 1)
        _, rot_preds = torch.max(rot_predictions, 1)

        # compute loss

        img_loss = criterion(imgs_predictions, lbls)
        rot_loss = criterion(rot_predictions, rot_lbls)

        loss = img_loss + args.weight_RotTask_step1*rot_loss

        # compute gradient + update params
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #statistics
        running_img_corrects += torch.sum(imgs_preds == lbls.data)
        running_rot_corrects += torch.sum(rot_preds == rot_lbls.data)

    img_acc = (running_img_corrects.double() / len(source_loader.dataset)) * 100
    rot_acc = (running_rot_corrects.double() / len(source_loader.dataset)) * 100

    return img_loss, img_acc, rot_loss, rot_acc

def step1(args,feature_extractor,rot_cls,obj_cls,source_loader,device):
    optimizer, scheduler = get_optim_and_scheduler(feature_extractor,rot_cls,obj_cls, args.epochs_step1, args.learning_rate, args.train_all)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(args.epochs_step1):
        print('Epoch: ',epoch)
        class_loss, acc_cls, rot_loss, acc_rot = _do_epoch(args,feature_extractor,rot_cls,obj_cls,source_loader,optimizer,device,criterion)
        print("Class Loss %.4f, Class Accuracy %.4f,Rot Loss %.4f, Rot Accuracy %.4f" % (class_loss.item(),acc_cls,rot_loss.item(), acc_rot))
        scheduler.step()
    
    torch.save(feature_extractor.state_dict(), "./feature_extractor_params.pt")
    torch.save(rot_cls.state_dict(), "./rot_cls_params.pt")
    torch.save(obj_cls.state_dict(), "./obj_cls_params.pt")

In [ ]:
#step2_SouceTargetAdaption

import torch
from torch import nn
from optimizer_helper import get_optim_and_scheduler
from itertools import cycle
import numpy as np


#### Implement Step2

def _do_epoch(args,feature_extractor,rot_cls,obj_cls,source_loader,target_loader_train,target_loader_eval,optimizer,device):

    criterion = nn.CrossEntropyLoss()
    feature_extractor.train()
    obj_cls.train()
    rot_cls.train()

    target_loader_train = cycle(target_loader_train)

    for it, (data_source, class_l_source, _, _) in enumerate(source_loader):

        (data_target, _, data_target_rot, rot_l_target) = next(target_loader_train)

        data_source, class_l_source  = data_source.to(device), class_l_source.to(device)
        data_target, data_target_rot, rot_l_target  = data_target.to(device), data_target_rot.to(device), rot_l_target.to(device)

        optimizer.zero_grad()


        class_loss = ....
        rot_loss = ....

        loss = class_loss + args.weight_RotTask_step2*rot_loss

        loss.backward()

        optimizer.step()

        _, cls_pred = ...
        _, rot_pred = ...

    acc_cls = ...
    acc_rot = ...

    print("Class Loss %.4f, Class Accuracy %.4f,Rot Loss %.4f, Rot Accuracy %.4f" % (class_loss.item(), acc_cls, rot_loss.item(), acc_rot))


    #### Implement the final evaluation step, computing OS*, UNK and HOS
    feature_extractor.eval()
    obj_cls.eval()
    rot_cls.eval()

    with torch.no_grad():
        for it, (data, class_l,_,_) in enumerate(target_loader_eval):



def step2(args,feature_extractor,rot_cls,obj_cls,source_loader,target_loader_train,target_loader_eval,device):
    optimizer, scheduler = get_optim_and_scheduler(feature_extractor,rot_cls,obj_cls, args.epochs_step2, args.learning_rate, args.train_all)


    for epoch in range(args.epochs_step2):
        _do_epoch(args,feature_extractor,rot_cls,obj_cls,source_loader,target_loader_train,target_loader_eval,optimizer,device)
        scheduler.step()